# Welcome!
This is a notebook detailing my analysis of some data about motor theft in New Zealand over 6 months.
The records contain information about the vehicle, including its unique ID, type, make, manufacture year, color the location of the incident, the day it was added to the database.

I figured it was an appropriate phenomenon to study, considering the imminent announcement of GTA VI.

The dataset is kindly provided by the wonderful people at [Maven Analytics](mavenanalytics.io), it can be acquired there for free.

## Goals:
### Are there seasonality trends to this phenomenon?
- I wonder if any crook would take the holidays as a chance to catch an owner off guard.
### Which vehicle types were stolen the most? 
- I expect standard cars to be the biggest victims, luxury cars tend to have more security features, and I don't think a common outlaw would want to accidentally steal a mob boss' car. Trailers are such are also prime candidates because you can just stick them to a tractor of any kind and run off with them, with the owner inside if you're feeling adventurous.
### What day of the week are vehicles most often and least often stolen?
- I'm quite curious about this one! If you stole a car, what day would you choose to do it?
### Is the model year a factor? Does it affect a vehicle differently based on its type?
- I expect prized cars from the 80s and earlier to be prime targets, highly valuable (particularly Japanese cars) without as much security as say, a 2023 Mercedes.
### Which regions had the most carjackings?
- I have no idea what the New Zealand provinces are, but a little filled map action will sort us well.
### Have carjackings increased or decreased in these six months?
- Analyzing the changes and trends of the phenomenon.
### Which regions are above or below the average rate of thefts?
- Using **transform()**, we'll be able to analyze whether a region is safer than others or not.



## Let's start by importing all the libraries we'll need:


In [17]:
# pandas
import pandas as pd

# numpy
import numpy as np

# matplotlib
import matplotlib.pyplot as plt

# seaborn, might need it
import seaborn as sns

# setting which plot style to use, I chose this one because a beige background is easier on the eyes, and it's viewable by colorblind people too
plt.style.use("Solarize_Light2")

#magic function that stores all the plots in the notebook
%matplotlib inline

## Data Import and Preprocessing

### Next, we'll import our CSV files and perform some import pre-processing to save memory and time:
- First we import the carjackings dataframe itself using **read_csv()**, set the date parser to true with specified date format, and use smaller integer data types when possible, these preprocessing steps will save us both time and performance.


In [18]:
# little variable to help make the code more readable
veh_cols=["vehicle_id","vehicle_type","make_id","model_year","color","date_stolen","location_id"]

thefts = pd.read_csv("E:/projects/nz-motor-theft-py-analysis/stolen_vehicles.csv",
            usecols=veh_cols, # this will pass only the specified columns
            index_col= "date_stolen", # setting the date column as an index
            header=0, # setting the first row as the header
            parse_dates=True, # allows pandas to directly make the date column a datetime64 type
            date_format= "%m/%d/%y", # helps pandas get the correct date format
            dtype={"vehicle_id": "Int16","make_id":"Int16","model_year":"Int16","location_id":"Int16"}, #notice the capital letter in the "Int"s, more on that later
            keep_default_na=True #keeps the NA values, so we'll be able to get rid of them later
            )

There's a lot of missing values here, and unfortunately we can't really do anything to fill them, so we'll have to get rid of them, but we'll save that for the cleaning stage.

- For now, we'll import the second CSV file, the make details table.

In [19]:
makes = pd.read_csv('E:/projects/nz-motor-theft-py-analysis/make_details.csv',
            index_col = "make_id",
            header=0, 
            dtype={"make_id":"Int16"},
            keep_default_na=True  
)
makes.head()

,make_name,make_type
make_id,,
501,Aakron Xpress,Standard
502,ADLY,Standard
503,Alpha,Standard
504,Anglo,Standard
505,Aprilia,Standard


What a pretty dataframe. We'll join it to the first table soon, using **merge()** - pandas' version of the SQL join, I love SQL, so I feel right at home.


- Now to import the location data:

In [20]:
location_cols=["location_id","region","population"] # columns to imported, there's only one country after all
locations = pd.read_csv('E:/projects/nz-motor-theft-py-analysis/locations.csv',
            index_col = "location_id",
            usecols=location_cols,
            header=0, 
            dtype={"location_id":"int8"},
            keep_default_na=True
  )
locations.head()

,region,population
location_id,,
101,Northland,"201,500"
102,Auckland,"1,695,200"
103,Waikato,"513,800"
104,Bay of Plenty,"347,700"
105,Gisborne,"52,100"


## Now that we have our data ready, it's time for cleaning.

- First, we'll check the stolen vehicles table:

In [21]:
thefts.shape

(4553, 6)

6 columns and 4527 rows, including the ones with missing data.

In [22]:
thefts.isna().sum() #first method will tell us if the value is missing or not


vehicle_id       0
vehicle_type    26
make_id         15
model_year      15
color           15
location_id      0
dtype: int64

Plenty of missing values. As mentioned before, we can't really fill them with anything, so we'll just get rid of any rows missing data using **dropna()**
Thankfully, it's only 26 rows. Negligible against the remaining 4520 (and header)

In [23]:
thefts = thefts.dropna(axis = 0, how = "any") #this will delete *any* rows that are missing values
thefts.isna().sum()

vehicle_id      0
vehicle_type    0
make_id         0
model_year      0
color           0
location_id     0
dtype: int64

Next, we'll check the makes table for any missing values, it's a lookup table, so I'm hoping nothing is missing.
Let's look at the shape:


In [24]:
makes.shape

(138, 2)

Let's check if any of these 138 rows are missing any data:


In [25]:
makes.isna().sum()

make_name    0
make_type    0
dtype: int64

None! What a relief.
Now, for the location table, it's likely the same, as it's a lookup table as well.

In [26]:
locations.shape

(16, 2)

In [27]:
locations.isna().sum()

region        0
population    0
dtype: int64

Success! There is no missing data in our lookup tables!

I noticed that the list of vehicle types is quite granular and could do with a little touch up. We'll use **contains()** nested in **where()** to replace any rows.


In [35]:
thefts_full["vehicle_type"].unique()

array(['Trailer', 'Boat Trailer', 'Roadbike', 'Moped', 'Trailer - Heavy',
       'Caravan', 'Hatchback', 'Saloon', 'Stationwagon', 'Tractor',
       'Trail Bike', 'Light Van', 'All Terrain Vehicle', 'Utility',
       'Other Truck', 'Sports Car', 'Flat Deck Truck', 'Light Bus',
       'Mobile Home - Light', 'Convertible', 'Heavy Van',
       'Special Purpose Vehicle', 'Articulated Truck',
       'Cab and Chassis Only', 'Mobile Machine'], dtype=object)

The way I see it, a trailer isn't that different to a boat trailer, nor is there a *huge* difference between a light and heavy van etc.

I apologize, it's quite an ugly snippet of code, and yes, I did try to use the **regex** library, but it doesn't support Dataframes and Series. So, I ended up making this:

In [ ]:
thefts_full['vehicle_type'] = thefts_full['vehicle_type'].where((thefts_full['vehicle_type'].str.contains('Trailer') == False), other="Trailer")
thefts_full['vehicle_type'] = thefts_full['vehicle_type'].where((thefts_full['vehicle_type'].str.contains('Truck') == False), other="Truck")
thefts_full['vehicle_type'] = thefts_full['vehicle_type'].where((thefts_full['vehicle_type'].str.contains('Van') == False), other="Van")thefts_full['vehicle_type'] = thefts_full['vehicle_type'].where((thefts_full['vehicle_type'].str.contains('Bus') == False), other="Bus")

It replaces any rows with the mention of a trailer with simply "trailer", van with "vans" and so on.
I spent over an hour trying to avoid that spaghetti but alas.

# Joining the Data tables:
For this, we'll use **merge()**, and NOT **join()** and keys like make_id and location_id.

Next, we'll join the location data, for the complete package:

In [30]:
thefts_full = thefts_w_makes.merge(locations,"left",left_on=thefts_w_makes["location_id"],right_index=True)
thefts_full

,vehicle_id,vehicle_type,make_id,model_year,color,location_id,make_name,make_type,region,population
date_stolen,,,,,,,,,,
2021-11-05,1,Trailer,623,2021,Silver,102,Trailer,Standard,Auckland,"1,695,200"
2021-12-13,2,Boat Trailer,623,2021,Silver,105,Trailer,Standard,Gisborne,"52,100"
2022-02-13,3,Boat Trailer,623,2021,Silver,102,Trailer,Standard,Auckland,"1,695,200"
2021-11-13,4,Trailer,623,2021,Silver,106,Trailer,Standard,Hawke's Bay,"182,700"
2022-01-10,5,Trailer,623,2018,Silver,102,Trailer,Standard,Auckland,"1,695,200"
...,...,...,...,...,...,...,...,...,...,...
2021-12-12,4523,Trailer,549,1993,Silver,102,Homebuilt,Standard,Auckland,"1,695,200"
2021-12-21,4524,Trailer,549,1994,Grey,103,Homebuilt,Standard,Waikato,"513,800"
2022-01-25,4525,Trailer,623,1995,Green,105,Trailer,Standard,Gisborne,"52,100"


Voilà! Now, we have the complete table, just like we would in SQL. We have the date of the theft, when and where it was committed, and which type of vehicle was stolen, etc. we are ready to aggregate!

# Aggregating and plotting the data:

In this step, we will:
- Aggregate the data by time, location, and make etc. to answer our questions.
- Use different visualizations to hopefully complete a sound and insightful analysis into this phenomenon.

### Which make types were stolen the most? 
To answer this question, it's a simple process of summing the thefts by make type, using **groupby()** and **sum()**

In [ ]:
thefts_w_makes.groupby()